In [1]:
import pandas as pd
import numpy as np
import os
import glob

np.random.seed(42)

In [2]:
# translate SNANA types
types_names = {90:'Ia', 67: '91bg', 52:'Iax', 42:'II', 62:'Ibc', 
               95: 'SLSN', 15:'TDE', 64:'KN', 88:'AGN', 92:'RRL', 65:'M-dwarf',
               16:'EB',53:'Mira', 6:'MicroL', 991:'MicroLB', 992:'ILOT', 
               993:'CART', 994:'PISN',995:'MLString'}

SNANA_types = {90:11, 62:{1:3, 2:13}, 42:{1:2, 2:12, 3:14},
               67:41, 52:43, 64:51, 95:60, 994:61, 992:62,
               993:63, 15:64, 88:70, 92:80, 65:81, 16:83,
               53:84, 991:90, 6:{1:91, 2:93}}

SNANA_names = {11: 'Ia', 3:'Ibc', 13: 'Ibc', 2:'II', 12:'II', 14:'II',
               41: '91bg', 43:'Iax', 51:'KN', 60:'SLSN', 61:'PISN', 62:'ILOT',
               63:'CART', 64:'TDE', 70:'AGN', 80:'RRL', 81:'M-dwarf', 83:'EB',
               84:'Mira', 90:'MicroLB', 91:'MicroL', 93:'MicroL'}

In [3]:
# read zenodo metadata
fname = '/media/RESSPECT/data/PLAsTiCC/PLAsTiCC_zenodo/plasticc_test_metadata.csv'
test_metadata = pd.read_csv(fname)

# separate fields
ddf_flag = test_metadata['ddf_bool'].values == 1
ids_ddf = test_metadata['object_id'].values[ddf_flag]
ids_wfd = test_metadata['object_id'].values[~ddf_flag]

# Create perfect samples

In [4]:
data_dir = '/media2/RESSPECT2/clean_output/'

## For DDF

In [5]:
# read all Ias in DDF
salt2_Ia_DDF = pd.read_csv('/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data/DDF/SALT2_fit/Ia/master_fitres.fitres', comment='#', delim_whitespace=True)
salt2_Ia_DDF['zHD'] = salt2_Ia_DDF['SIM_ZCMB']          # requirement of so SALT2mu can work

# choose sample size
nobjs = 3000

# choose number of versions of the same sample to generate
v = 6

for i in range(v):
    perfect_Ia_DDF = salt2_Ia_DDF.sample(n=nobjs, replace=False)
    #perfect_Ia_DDF.to_csv(data_dir + 'DDF/v' + str(i) +  '/samples/perfect' + \
    #                      str(nobjs) + '.csv', sep=' ', index=False)

## For WFD

In [8]:
# read all Ias in WFD
fnames_Ia = glob.glob('/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data/WFD/SALT2_fit/Ia/master_fitres_*.fitres')

salt2_WFD = []

for name in fnames_Ia:
    fitres_temp = pd.read_csv(name, delim_whitespace=True, 
                              comment='#')
    fitres_temp['zHD'] = fitres_temp['SIM_ZCMB']
    salt2_WFD.append(fitres_temp)

salt2_Ia_WFD = pd.concat(salt2_WFD, ignore_index=True)

# choose sample size
nobjs = 3000

# choose number of versions of the same sample to generate
v = 6

for i in range(v):
    perfect_Ia_WFD = salt2_Ia_WFD.sample(n=nobjs, replace=False)
    #perfect_Ia_WFD.to_csv(data_dir + 'WFD/v' + str(i) + '/samples/perfect' + \
    #                      str(nobjs) + '.csv', sep=' ', index=False)

# Create Random samples

## For DDF

In [10]:
# list of classes surviving SALT2 fit
surv_class_DDF = ['91bg', 'AGN', 'CART', 'Ia', 'Iax', 'Ibc', 'II', 'TDE']

# read all SALT2 fit results for DDF
all_DDF = []
for obj_type in surv_class_DDF:
    data_temp = pd.read_csv('/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data/DDF/SALT2_fit/' + obj_type + '/master_fitres.fitres', 
                            comment='#', delim_whitespace=True)
    data_temp['zHD'] = data_temp['SIM_ZCMB']
    data_temp.fillna(-99, inplace=True)
    all_DDF.append(data_temp)
    
all_surv_DDF = pd.concat(all_DDF, ignore_index=True)
all_surv_DDF.fillna(-99, inplace=True)

# choose sample size
nobjs = 3000

# choose number of versions of the same sample to generate
v = 6

for i in range(v):
    random_DDF = all_surv_DDF.sample(n=nobjs, replace=False)
    #random_DDF.to_csv(data_dir + 'DDF/v' + str(i) + '/samples/random' + \
    #                  str(nobjs) + '.csv', sep=' ', index=False)

In [32]:
snana_numbers_ddf, freq_ddf = np.unique(all_surv_DDF['SIM_TYPE_INDEX'].values, 
                                        return_counts=True)

for i in range(len(freq_ddf)):
    print(SNANA_names[snana_numbers_ddf[i]], '\t', freq_ddf[i], 
          '\t', 100*freq_ddf[i]/all_surv_DDF.shape[0])

II 	 525 	 4.413990247183454
Ibc 	 158 	 1.328400874390449
Ia 	 10327 	 86.82529006221624
II 	 262 	 2.202791323356314
Ibc 	 93 	 0.7819068437867832
II 	 63 	 0.5296788296620145
91bg 	 6 	 0.05044560282495376
Iax 	 428 	 3.598453001513368
CART 	 28 	 0.23541281318311755
TDE 	 1 	 0.008407600470825626
AGN 	 3 	 0.02522280141247688


In [36]:
# type II 
100*(525+262+63)/all_surv_DDF.shape[0]

7.146460400201782

In [42]:
# type Ibc
100*(158+93)/all_surv_DDF.shape[0]

2.110307718177232

In [43]:
all_surv_DDF.shape

(11894, 100)

## For WFD

In [13]:
# list of classes surviving SALT2 fit
surv_class_WFD = ['91bg', 'AGN', 'CART', 'Ia', 'Iax', 'Ibc', 'II', 'TDE', 'ILOT', 'PISN', 'SLSN']

# read all SALT2 fit results for WFD
all_WFD = []
for obj_type in surv_class_WFD:
    flist = glob.glob('/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data/WFD/SALT2_fit/' + obj_type + '/master_fitres_*.fitres')
    
    for name in flist:
        data_temp = pd.read_csv(name, comment='#', delim_whitespace=True)
        data_temp['zHD'] = data_temp['SIM_ZCMB']
        data_temp.fillna(-99, inplace=True)
        all_WFD.append(data_temp)
        
    
all_surv_WFD = pd.concat(all_WFD, ignore_index=True)
all_surv_WFD.fillna(-99, inplace=True)

# choose sample size
nobjs = 3000

# choose number of versions of the same sample to generate
v = 6

for i in range(6, v):
    random_WFD = all_surv_WFD.sample(n=nobjs, replace=False)
    #random_WFD.to_csv(data_dir + 'WFD/v' + str(i) + '/samples/perfect' + \
    #                  str(nobjs) + '.csv', sep=' ', index=False)

In [14]:
salt2_Ia_WFD.shape[0]/all_surv_WFD.shape[0]

0.9073704700336027

In [31]:
snana_numbers_wfd, freq_wfd = np.unique(all_surv_WFD['SIM_TYPE_INDEX'].values, return_counts=True)

for i in range(len(freq_wfd)):
    print(SNANA_names[snana_numbers_wfd[i]], '\t', freq_wfd[i], '\t', 100*freq_wfd[i]/all_surv_WFD.shape[0])

II 	 31414 	 3.021163726031402
Ibc 	 3558 	 0.3421818468587168
Ia 	 943482 	 90.73704700336027
II 	 26065 	 2.506736885433517
Ibc 	 2508 	 0.24120069475032652
II 	 4854 	 0.46682144031821565
91bg 	 881 	 0.08472799524523032
Iax 	 23698 	 2.27909651682346
SLSN 	 529 	 0.0508752661574652
ILOT 	 18 	 0.001731105464715262
CART 	 1349 	 0.12973673732782715
TDE 	 152 	 0.014618223924262213
AGN 	 1290 	 0.12406255830459377


In [26]:
# type II
100*(31414+ 26065+4854)/all_surv_WFD.shape[0]

5.9947220517831346

In [30]:
# type Ibc
100*(3558+2508)/all_surv_WFD.shape[0]

0.5833825416090433

In [44]:
all_surv_WFD.shape

(1039798, 110)

# Create Fiducial samples

## For DDF

In [37]:
# read results from avocado
fname_DDF = data_dir + 'DDF/avocado/avocado_DDF.csv'
avocado_DDF = pd.read_csv(fname_DDF, names=['object_id','6','15','16','42','52','53','62','64','65','67','88',
                                           '90','92','95'], skiprows=1)

# determine final classification
class_final_DDF = []
for i in range(avocado_DDF.shape[0]):
    indx = np.argsort(avocado_DDF.iloc[i].values[1:])[-1]
    code = int(avocado_DDF.keys()[indx + 1])
    class_final_DDF.append(types_names[code])
class_final_DDF = np.array(class_final_DDF)

# get photometrically classified Ia
flag_class_Ia_DDF = class_final_DDF == 'Ia'
avocado_DDF_Ia = avocado_DDF[flag_class_Ia_DDF]

# get SALT2 fit for objs photometrically classified as Ia
avocado_DDF_Ia_fitres_flag = np.array([item in avocado_DDF_Ia['object_id'].values for item in all_surv_DDF['CID'].values])
all_avocado_DDF_Ia = all_surv_DDF[avocado_DDF_Ia_fitres_flag]

# choose sample size
nobjs = 6000

# choose number of versions of the same sample to generate
v = 6

for i in range(v):
    fiducial_DDF = all_avocado_DDF_Ia.sample(n=nobjs, replace=False)
    #fiducial_DDF.to_csv(data_dir + 'DDF/v' + str(i) + '/samples/fiducial' + \
    #                  str(nobjs) + '.csv', sep=' ', index=False)

## For WFD

In [23]:
# read results from avocado
fname_WFD =  data_dir + 'WFD/avocado/avocado_WFD.csv'
avocado_WFD = pd.read_csv(fname_WFD, names=['object_id','6','15','16','42','52','53','62','64','65','67','88',
                                           '90','92','95'], skiprows=1)

# determine final classification
class_final_WFD = []
for i in range(avocado_WFD.shape[0]):
    indx = np.argsort(avocado_WFD.iloc[i].values[1:])[-1]
    code = int(avocado_WFD.keys()[indx + 1])
    class_final_WFD.append(types_names[code])
    
# get photometrically classified Ia
class_final_WFD = np.array(class_final_WFD)
flag_class_Ia_WFD = class_final_WFD == 'Ia'
avocado_WFD_Ia = avocado_WFD[flag_class_Ia_WFD]

# get SALT2 fit for objs photometrically classified as Ia
avocado_WFD_Ia_fitres_flag = np.array([item in avocado_WFD_Ia['object_id'].values 
                                       for item in all_surv_WFD['CID'].values])
all_avocado_WFD_Ia = all_surv_WFD[avocado_WFD_Ia_fitres_flag]

# choose sample size
nobjs = 3000

# choose number of versions of the same sample to generate
v = 6

for i in range(v):
    fiducial_WFD = all_avocado_WFD_Ia.sample(n=nobjs, replace=False)
    #fiducial_WFD.to_csv(data_dir + 'WFD/v' + str(i) + '/samples/perfect' + \
    #                  str(nobjs) + '.csv', sep=' ', index=False)

# Create single contaminant samples

## For DDF

In [39]:
# levels of contamination
cont_DDF = {'II': [0.28, 0.25, 0.1, 0.05, 0.02, 0.01],
            'Ibc': [0.05, 0.02, 0.01],
            'Iax': [0.14, 0.1, 0.05, 0.02, 0.01],
            'CART': [0.009], 
            '91bg': [0.002],
            'AGN': [0.001]}

complete_names ={'II': 'SNII', 'Ibc': 'SNIbc', 'Iax': 'SNIax', 'CART':'CART',
                 '91bg':'SNIa-91bg', 'AGN':'AGN'}

# choose sample size
nobjs = 3000

# choose number of versions of the same sample to generate
v = 6

for i in range(v):
    for obj_class in list(cont_DDF.keys()):
        # read all contaminants surviving SALT2 fit
        sample_cont = pd.read_csv('/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data/DDF/SALT2_fit/' + obj_class + '/master_fitres.fitres', comment='#',
                              delim_whitespace=True)
        sample_cont['zHD'] = sample_cont['SIM_ZCMB']
    
        for perc in cont_DDF[obj_class]:
            Ia_temp = salt2_Ia_DDF.sample(n=int((1 - perc) * nobjs), replace=False)
            cont_temp = sample_cont.sample(n=int(perc * nobjs), replace = False)
            sample_final = pd.concat([Ia_temp, cont_temp], ignore_index=True)
            sample_final.fillna(-99, inplace=True)
        
            if obj_class not in ['CART', '91bg', 'AGN']:
                sample_final.to_csv(data_dir + 'DDF/v' + str(i) + '/samples/' + str(int(100 - 100 * perc)) + \
                                    'SNIa' + str(int(100 * perc)) + complete_names[obj_class] + '.csv', 
                                     sep=' ', index=False)
            else:
                sample_final.to_csv(data_dir + 'DDF/v' + str(i) + '/samples/' + str(round(100 - 100 * perc, 1)) + \
                                    'SNIa' + str(round(100 * perc, 1)) + complete_names[obj_class] + '.csv', 
                                     sep=' ', index=False)

## For WFD

In [45]:
# levels of contamination
cont_WFD = {'II': [0.28, 0.25, 0.1, 0.05, 0.02, 0.01],
            'Ibc': [0.1, 0.05, 0.02, 0.01],
            'Iax': [0.25, 0.1, 0.05, 0.02, 0.01],
            '91bg': [0.05, 0.02, 0.01],
            'AGN': [0.05, 0.02, 0.01],
            'TDE': [0.004],
            'CART': [0.003]}

complete_names ={'II': 'SNII', 'Ibc': 'SNIbc', 'Iax': 'SNIax', 'CART':'CART',
                 '91bg':'SNIa-91bg', 'AGN':'AGN', 'TDE':'TDE'}

# choose sample size
nobjs = 3000

# choose number of versions of the same sample to generate
v = 6

for i in range(v):
    for obj_class in list(cont_WFD.keys()):
        # read all contaminants surviving SALT2 fit
        flist = glob.glob('/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data/WFD/SALT2_fit/' + obj_class + '/master_fitres_*.fitres')
        
        sample_cont = []
        for name in flist:
            temp_cont = pd.read_csv(name, comment='#', delim_whitespace=True)
            temp_cont['zHD'] = temp_cont['SIM_ZCMB']
            sample_cont.append(temp_cont)
            
        sample_cont2 = pd.concat(sample_cont, ignore_index=True)
    
        for perc in cont_WFD[obj_class]:
            Ia_temp2 = salt2_Ia_WFD.sample(n=int((1-perc)*nobjs), replace=False)
            cont_temp2 = sample_cont2.sample(n=int(perc*nobjs), replace = False)
            sample_final = pd.concat([Ia_temp2, cont_temp2], ignore_index=True)
            sample_final.fillna(-99, inplace=True)
        
            #sample_final.to_csv(data_dir + 'WFD/v' + str(i) + '/samples/' + str(int(100 - 100 * perc)) + \
            #              'SNIa' + str(int(100 * perc)) + complete_names[obj_class] + '.csv', 
            #              sep=' ', index=False)